In [1]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from xarray import DataArray
import pandas as pd
from PyEMD import EEMD
import pylab as plt

In [2]:
mode = 4+1 # mode(input, c1-c4)
time = 50
ly = 10
hind_eemd_9 = np.zeros((5,50,10)) # mode(input, c1-c4), time, lead year


In [3]:
data_h       = xr.open_dataset("/work/uo1075/u241321/data/thetao_ly_1970-2019_hindcast80.nc",decode_times=False) 
lat_min, lat_max = (60,76) # Norwegian Sea
lon_min, lon_max = (0,20)



In [6]:
# weight average for hindcast
var22 = data_h['thetao'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))


In [9]:
v1

<xarray.DataArray 'thetao' (time: 50, lead_year: 10, depth: 21, lat: 16, lon: 20)>
array([[[[[-5.09119213e-01, -5.24756372e-01, -5.35131812e-01, ...,
                       nan, -8.55555415e-01, -8.58107209e-01],
          [-4.69150126e-01, -4.88775671e-01, -5.57187021e-01, ...,
                       nan, -8.48557830e-01, -7.08249450e-01],
          [-6.49099886e-01, -5.92589080e-01, -5.31275809e-01, ...,
                       nan,             nan, -6.86582804e-01],
          ...,
          [-1.48487461e+00, -1.47249103e+00, -1.41843104e+00, ...,
           -3.80863667e-01, -4.67125803e-01, -4.97997195e-01],
          [-1.26389813e+00, -1.23367429e+00, -1.18064857e+00, ...,
           -5.96586406e-01, -6.96121752e-01, -6.35327220e-01],
          [-1.21958947e+00, -1.32144463e+00, -1.42861509e+00, ...,
           -8.01089644e-01, -7.34938920e-01, -6.28064275e-01]],

         [[-4.97700363e-01, -4.98769820e-01, -5.37931442e-01, ...,
                       nan, -8.40318203e-01, -8.50141644e-01],
          [-4.62233454e-01, -4.59176958e-01, -5.71000457e-01, ...,
                       nan, -8.40484321e-01, -6.98936760e-01],
          [-6.46710694e-01, -5.91642857e-01, -5.28863788e-01, ...,
                       nan,             nan, -6.79301620e-01],
...
                       nan,             nan,             nan],
          [ 2.69475043e-01,  2.73534954e-01,  2.85783708e-01, ...,
                       nan,             nan,             nan],
          [ 2.79750556e-01,  2.40370721e-01,  1.90678895e-01, ...,
                       nan,             nan,             nan]],

         [[            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [            nan,             nan,             nan, ...,
                       nan,             nan,             nan],
          [ 8.52405578e-02,             nan,             nan, ...,
                       nan,             nan,             nan],
          ...,
          [ 6.86415061e-02,  8.79542902e-02,  1.30656615e-01, ...,
                       nan,             nan,             nan],
          [ 1.64048731e-01,  1.58502370e-01,  1.65986031e-01, ...,
                       nan,             nan,             nan],
          [ 2.00027138e-01,  1.67351350e-01,  1.53960869e-01, ...,
                       nan,             nan,             nan]]]]],
      dtype=float32)
Coordinates:
  * lead_year  (lead_year) int64 1 2 3 4 5 6 7 8 9 10
  * time       (time) float64 115.5 127.5 139.5 151.5 ... 679.5 691.5 703.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 15.5 16.5 17.5 18.5 19.5
  * lat        (lat) float64 60.5 61.5 62.5 63.5 64.5 ... 72.5 73.5 74.5 75.5
  * depth      (depth) float64 6.0 17.0 27.0 37.0 ... 485.0 560.0 645.0 740.0

In [8]:
v1=np.mean(var22, axis=1)

In [10]:
sst_h = np.mean(v1[:,:,0:15,:,:],2)


In [11]:
var2 = sst_h

In [12]:
weights = np.cos(np.deg2rad(var2.lat))
weights.name = "weights"
air_weighted1 = var2.weighted(weights)
hind_eemd_9[0,:,:] = air_weighted1.mean(("lon", "lat"))

In [13]:
# EEMD options
max_imf=3
# Prepare and run EEMD
eemd = EEMD(trials=10000, FIXE_H=10, DTYPE=np.float16, noise_width=0.2)   #max_imfs=10)
eemd.noise_seed(6021)

t = np.arange(1970,2020,1)

In [14]:
# Execute EEMD on Hindcast                
for i in range(0,ly,1):                       
    hind_eemd_9[1::,:,i] = eemd.eemd(hind_eemd_9[0,:,i], t, max_imf)  
           

In [15]:
np.save("/work/uo1075/u241321/data/hind_eemd_t_80", hind_eemd_9)


In [16]:
hind_eemd_9

array([[[-4.56476972e-01, -5.48618726e-01, -3.43332836e-01, ...,
         -5.95244739e-01, -6.62528969e-01, -7.20249215e-01],
        [-6.22887018e-01, -4.38306271e-01, -4.46639259e-01, ...,
         -6.41959437e-01, -5.86790187e-01, -6.49780886e-01],
        [-3.21738904e-01, -5.36165438e-01, -4.31280910e-01, ...,
         -6.29174660e-01, -6.06668722e-01, -5.48151460e-01],
        ...,
        [ 9.14992338e-01,  5.26104833e-01,  4.02739548e-01, ...,
          4.87754717e-01,  4.88493915e-01,  4.61140619e-01],
        [ 9.03404631e-01,  7.41775645e-01,  4.64942057e-01, ...,
          3.72678690e-01,  4.29426003e-01,  5.30699115e-01],
        [ 5.57844746e-01,  6.61343082e-01,  6.38302694e-01, ...,
          4.06030035e-01,  4.54123091e-01,  4.32591383e-01]],

       [[-2.96590786e-02, -5.55975876e-02,  4.71445501e-02, ...,
          2.60471671e-02, -3.05721292e-02, -6.40871470e-02],
        [-1.56513745e-01,  4.91386292e-02, -2.80475213e-02, ...,
         -3.40788651e-03,  2.60276782e